# Keras model fitting

This notebook trains a simple neural net using Keras and assesses its performance.

In [ ]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

This notebook is parameterized to work with [Papermill](https://papermill.readthedocs.io).
The following cell contains the default values of the parameters.

In [ ]:
log_dir = "logs"

n_units = 10
n_epochs = 2
n_batch = 32
verbose = 1

First we load the training dataset.

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print(f"Number of training samples: {len(X_train)}.")
print(f"Number of test samples: {len(X_test)}.")

This dataset contains images of digits. Here is a sample.

In [ ]:
_, axes = plt.subplots(1, 10, figsize=(12, 5))
for ax, digit in zip(axes, X_train):
    ax.imshow(digit)
    ax.axis("off")

Next, we transform the data to be more easily processed by the Keras neural net.

In [ ]:
n_classes = 10
y_train = tf.one_hot(y_train, n_classes)
y_test = tf.one_hot(y_test, n_classes)

X_train = X_train.astype(np.float32) / 255.
X_test = X_test.astype(np.float32) / 255.

Then define a MLP model, using Keras.

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
        tf.keras.layers.Dense(n_units, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(n_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.01),
    metrics=["accuracy"],
)

Fitting will take more or less time depending on the total number of epochs used.

In [ ]:
log_dir = log_dir + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
%%time
history = model.fit(
    X_train,
    y_train,
    epochs=n_epochs,
    batch_size=n_batch,
    verbose=verbose,
    callbacks=[tensorboard_callback],
)

Finally, we check the accuracy on the test dataset.

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print(f"MLP test accuracy is {accuracy * 100:.2f}%.")

We can investigate the results via Tensorboard, use the integrated reverse proxy to access it.

In [ ]:
%env TENSORBOARD_PROXY_URL /user-redirect/proxy/%PORT%/

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs